In [1]:
import re

In [2]:
import numpy as np
import pandas as pd

In [3]:
from pprint import pprint

In [5]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [6]:
from nltk.corpus import stopwords

In [7]:
import spacy

In [11]:
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [17]:
df = pd.read_json('C:\\Users\\ergarciap\\Desktop\\ERGARCIAP\\Erick\\MASTER_UVG\\DataMining_NaturalLenguage\\Tarea 3\\Estructura_predict_temas_correos\\data\\raw\\newsgroups.json')
print(df.target_names.unique())
df

['rec.autos' 'comp.sys.mac.hardware' 'comp.graphics' 'sci.space'
 'talk.politics.guns' 'sci.med' 'comp.sys.ibm.pc.hardware'
 'comp.os.ms-windows.misc' 'rec.motorcycles' 'talk.religion.misc'
 'misc.forsale' 'alt.atheism' 'sci.electronics' 'comp.windows.x'
 'rec.sport.hockey' 'rec.sport.baseball' 'soc.religion.christian'
 'talk.politics.mideast' 'talk.politics.misc' 'sci.crypt']


,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space
...,...,...,...
11309,From: jim.zisfein@factory.com (Jim Zisfein) \n...,13,sci.med
11310,From: ebodin@pearl.tufts.edu\nSubject: Screen ...,4,comp.sys.mac.hardware
11311,From: westes@netcom.com (Will Estes)\nSubject:...,3,comp.sys.ibm.pc.hardware
11312,From: steve@hcrlgw (Steven Collins)\nSubject: ...,1,comp.graphics


In [18]:
# Convertir a una lista
data = df.content.values.tolist()
pprint(data[:1])

["From: lerxst@wam.umd.edu (where's my thing)\n"
 'Subject: WHAT car is this!?\n'
 'Nntp-Posting-Host: rac3.wam.umd.edu\n'
 'Organization: University of Maryland, College Park\n'
 'Lines: 15\n'
 '\n'
 ' I was wondering if anyone out there could enlighten me on this car I saw\n'
 'the other day. It was a 2-door sports car, looked to be from the late 60s/\n'
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition,\n'
 'the front bumper was separate from the rest of the body. This is \n'
 'all I know. If anyone can tellme a model name, engine specs, years\n'
 'of production, where this car is made, history, or whatever info you\n'
 'have on this funky looking car, please e-mail.\n'
 '\n'
 'Thanks,\n'
 '- IL\n'
 '   ---- brought to you by your neighborhood Lerxst ----\n'
 '\n'
 '\n'
 '\n'
 '\n']


In [19]:
# Eliminar emails - Todo lo que este antes y despues del arroba
data = [re.sub(r'\S*@\S*\s?', '', sent) for sent in data]
pprint(data[:1])

["From: (where's my thing)\n"
 'Subject: WHAT car is this!?\n'
 'Nntp-Posting-Host: rac3.wam.umd.edu\n'
 'Organization: University of Maryland, College Park\n'
 'Lines: 15\n'
 '\n'
 ' I was wondering if anyone out there could enlighten me on this car I saw\n'
 'the other day. It was a 2-door sports car, looked to be from the late 60s/\n'
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition,\n'
 'the front bumper was separate from the rest of the body. This is \n'
 'all I know. If anyone can tellme a model name, engine specs, years\n'
 'of production, where this car is made, history, or whatever info you\n'
 'have on this funky looking car, please e-mail.\n'
 '\n'
 'Thanks,\n'
 '- IL\n'
 '   ---- brought to you by your neighborhood Lerxst ----\n'
 '\n'
 '\n'
 '\n'
 '\n']


In [20]:
# Eliminar newlines
data = [re.sub(r'\s+', ' ', sent) for sent in data]
pprint(data[:1])

["From: (where's my thing) Subject: WHAT car is this!? Nntp-Posting-Host: "
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


In [21]:
# Eliminar comillas
data = [re.sub(r"\'", "", sent) for sent in data]
pprint(data[:1])

['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


In [22]:
def sent_to_words(sentences):
    for sentence in sentences:
    # https://radimrehurek.com/gensim/utils.html#gensim.utils.simple_preprocess
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True)) 
    # deacc=True elimina la puntuación
data_words = list(sent_to_words(data))

print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


In [23]:
# Construimos modelos de bigrams y trigrams
# https://radimrehurek.com/gensim/models/phrases.html#gensim.models.phrases.Phrases
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Aplicamos el conjunto de bigrams/trigrams a nuestros documentos
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [24]:
print(bigram_mod[data_words[0]])
print(trigram_mod[bigram_mod[data_words[0]]])

['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting', 'host', 'rac_wam', 'umd_edu', 'organization', 'university', 'of', 'maryland_college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']
['from', 'wheres', 'my', 'thing', 'subject', '

In [25]:
# Eliminar stopwords
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# Hacer bigrams
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

# Hacer trigrams
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

# Lematización basada en el modelo de POS de Spacy
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [26]:
# Eliminamos stopwords
data_words_nostops = remove_stopwords(data_words)

In [27]:
# Formamos bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

In [28]:
# python3 -m spacy download en_core_web_lg
# Inicializamos el modelo 'en_core_web_lg' con las componentes de POS únicamente
nlp = spacy.load('C:\\Users\\ergarciap\\Miniconda3\\Lib\\site-packages\\en_core_web_lg\\en_core_web_lg-2.3.0', disable=['parser', 'ner'])

# Lematizamos preservando únicamente noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['where', 'car', 'nntp_poste', 'host', 'rac_wam', 'park', 'line', 'wonder', 'could', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'door', 'really', 'small', 'addition', 'separate', 'rest', 'body', 'know', 'model', 'name', 'engine', 'spec', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood']]


In [29]:
# Creamos diccionario
id2word = corpora.Dictionary(data_lemmatized)

In [30]:
for key, value in id2word.items():
    print(key, value)

0 addition
1 body
2 bring
3 call
4 car
5 could
6 day
7 door
8 early
9 engine
10 enlighten
11 funky
12 history
13 host
14 info
15 know
16 late
17 line
18 look
19 mail
20 make
21 model
22 name
23 neighborhood
24 nntp_poste
25 park
26 production
27 rac_wam
28 really
29 rest
30 see
31 separate
32 small
33 spec
34 sport
35 thank
36 where
37 wonder
38 year
39 acceleration
40 adapter
41 add
42 answer
43 article
44 attain
45 base
46 brave
47 brief
48 card
49 clock
50 detail
51 do
52 especially
53 experience
54 fair
55 final
56 floppy
57 functionality
58 guy_kuo
59 heat_sink
60 hour
61 keyword
62 knowledge
63 message
64 network
65 next
66 number
67 oscillator
68 poll
69 procedure
70 rate
71 report
72 request
73 send
74 share
75 si
76 soul
77 speed
78 summarize
79 summary
80 top
81 upgrade
82 usage
83 access
84 active
85 actually
86 advance
87 anymore
88 appearence
89 back
90 be
91 bite
92 breifly
93 bunch
94 computer
95 conviction
96 daily
97 dirt
98 disk
99 display
100 distribution_usa
101 dro

1617 err
1618 excellent
1619 factory
1620 fangled
1621 fatal
1622 firearm
1623 fit
1624 fragile
1625 function
1626 gun
1627 hammer
1628 hurt
1629 iftccu_talk
1630 imagine
1631 indifferent
1632 issue
1633 jammed
1634 loader
1635 locked
1636 lover
1637 majority
1638 malfunction
1639 maximum
1640 merely
1641 misfire
1642 motion
1643 necessary
1644 notoriously
1645 often
1646 open
1647 operation
1648 paragraph
1649 pistol
1650 police
1651 politics_gun
1652 pound
1653 primer
1654 pull
1655 quality
1656 redundant
1657 reload
1658 resist
1659 revolver
1660 self_defense
1661 semi_autos
1662 series
1663 shell
1664 shoot
1665 slide
1666 somewhere
1667 speedloader
1668 technical
1669 tie
1670 train
1671 training
1672 trigger
1673 unnecessarily
1674 user
1675 vital
1676 wear
1677 affect
1678 analyze
1679 assist
1680 athlete
1681 ban
1682 baseball
1683 basis
1684 biased
1685 consequence
1686 criticize
1687 derogatory
1688 downright
1689 faggot
1690 game
1691 hire
1692 honestly
1693 ludicrous
1694 m

2960 initialize
2961 int
2962 menu
2963 overload
2964 overloaded
2965 piece
2966 portability
2967 properly
2968 void
2969 whole
2970 arbitrary
2971 char
2972 color
2973 constrain
2974 convex
2975 dept
2976 depth
2977 dresden_de
2978 extrude
2979 generic
2980 hull
2981 mirror
2982 picture
2983 placement
2984 polygonal
2985 postscript
2986 pov
2987 previewer
2988 rayshade
2989 roman
2990 scale
2991 selfmade
2992 shape
2993 stroke
2994 texture
2995 thick
2996 translate
2997 triangulate
2998 union
2999 upwards
3000 devil
3001 goalie
3002 helmet
3003 mould
3004 remind
3005 retire
3006 scorer
3007 valerie_hammerl
3008 assembler
3009 compilier
3010 family
3011 impor
3012 out
3013 processor
3014 propreitary
3015 related
3016 responce
3017 rthe
3018 tant
3019 admonition
3020 befall
3021 chronicle
3022 collaboration
3023 conceal
3024 definite
3025 economic
3026 figment
3027 heap
3028 historical
3029 imply
3030 nazi_germany
3031 proof
3032 publish
3033 wipe
3034 bang
3035 buffer
3036 controller
3

4526 luckily
4527 manufacture
4528 mature
4529 moderator
4530 monk
4531 mortal
4532 mostly
4533 muster
4534 mysterious
4535 netter
4536 nuke
4537 objectively
4538 oddity
4539 persist
4540 pointless
4541 pyramid
4542 sence
4543 speil
4544 spirit
4545 suffice
4546 sun_microsystems
4547 surely
4548 transcend
4549 ulitmately
4550 unreachable
4551 utterly
4552 classical
4553 communicate
4554 incompatible
4555 incomplete
4556 k_p
4557 protocol
4558 session_key
4559 aweto
4560 bob
4561 economy
4562 funding_cuts
4563 shut
4564 communication
4565 grab
4566 internally
4567 interrupt
4568 loose
4569 uart
4570 carpet
4571 drain_hole
4572 gasket
4573 gradually
4574 hatch
4575 plywood
4576 scoop
4577 storm
4578 trunk
4579 autograph
4580 buddy
4581 coach
4582 distribution_na
4583 finished
4584 firepower
4585 goaltender
4586 handed
4587 immaterial
4588 inept
4589 interview
4590 irrelevent
4591 judge
4592 low_gaa
4593 masterpiece
4594 mistake
4595 opponet
4596 outperform
4597 photo
4598 pizza_hut
4599 

6116 centre
6117 circle
6118 coincident
6119 collinear
6120 coplaner
6121 ed
6122 embarasse
6123 eq
6124 equi
6125 equidistant
6126 geometry
6127 graphics_gem
6128 hyper
6129 numerically
6130 perpendicular
6131 sphere
6132 straightforward
6133 bank_switche
6134 calculate
6135 optimistically
6136 quadruple
6137 whit
6138 ega
6139 vga_monitor
6140 adobe_system
6141 believable
6142 db
6143 flamebait
6144 guise
6145 jet
6146 jumbo
6147 lab_mix
6148 maxim_dod
6149 rpm
6150 threshold
6151 frontal
6152 lend
6153 lobe
6154 lobotomy
6155 notional
6156 partition
6157 phenomenon
6158 prey
6159 razor
6160 satisfactorily
6161 scientist
6162 simple_minde
6163 straighten
6164 theist
6165 unscientific
6166 commemorative
6167 cousin
6168 cultural
6169 grandparent
6170 nazi
6171 painfully
6172 restitution
6173 th_anniversary
6174 timelessness
6175 unite
6176 doco
6177 mfm_rll
6178 ac_uk
6179 raytracer
6180 raytracing
6181 synchrotron
6182 visualization
6183 fetish
6184 fetishism
6185 immoral
6186 monoga

...

26237 resurgence
26238 uimx
26239 visix
26240 inderectly
26241 japaneese
26242 clayton_brook
26243 mutt
26244 trifle
26245 corinthian
26246 gifted
26247 tounge
26248 toungue
26249 fortify
26250 ghostview
26251 pageview
26252 bicyclist
26253 fearing
26254 provious
26255 safety_deserve
26256 midweek
26257 scoresheet
26258 mammal
26259 unprovable
26260 panther
26261 potvin
26262 bodycount
26263 clusterf
26264 shortdrive
26265 tramping
26266 prisioner
26267 eighty
26268 clip_x_origin
26269 fg_color
26270 stipple
26271 w_height
26272 w_id
26273 xcopyplane
26274 reborn
26275 ope
26276 otaku
26277 advthanksance
26278 afffff
26279 affraid
26280 manupulation
26281 arbitrator
26282 chevrolet
26283 repurchase
26284 unprepared
26285 nestvx
26286 coloured
26287 fleetway
26288 marvin_batty
26289 appleshare
26290 layeth
26291 ballsy
26292 bigger
26293 bream
26294 chronological
26295 cigar
26296 credibly
26297 flounder
26298 honeymoon
26299 mariner
26300 posterity
26301 quantrill
26302 stathead
26303 

27803 kenworth
27804 multicylinder
27805 whiny
27806 har_har
27807 ella_baff
27808 john_badane
27809 cy_young
27810 john_hawkinson
27811 boastful
27812 canoncial
27813 codex
27814 disqulaify
27815 episcop
27816 equivocal
27817 faithless
27818 instuctor
27819 misatribute
27820 provocatively
27821 belliard
27822 knuckler
27823 offenivsively
27824 offside
27825 conserned
27826 resistence
27827 biophysic
27828 reseach
27829 reseacher
27830 castor
27831 hhhmmmmm
27832 peeing
27833 polyester
27834 skys
27835 governance
27836 wholely
27837 leatherette
27838 roamer
27839 takumar
27840 exit_code
27841 revver
27842 rockwell_international
27843 belting
27844 boranni
27845 innertube
27846 alveoli
27847 alveolus
27848 oan
27849 dblspace
27850 exitte
27851 cnter
27852 determininant
27853 fearly
27854 hardpressed
27855 ozolinshs
27856 zettler
27857 zmolek
27858 frisbee
27859 antiserie
27860 elcos
27861 reinveste
27862 hhy
27863 blocking
27864 worldbank
27865 f
27866 slighty
27867 mmle
27868 adbs
2786

29432 reconstitute
29433 select_militia
29434 sinew
29435 skirt
29436 unorganized
29437 plotdoc
29438 plplot
29439 divluge
29440 tryie
29441 scott_roby
29442 sigpipe
29443 contadictorily
29444 contradictorily
29445 edict
29446 particlar
29447 ta
29448 happend
29449 soundbase
29450 technoculture
29451 brainless
29452 clod
29453 bone_marrow
29454 geneticist
29455 gillow
29456 headpiece
29457 heft
29458 innermost
29459 inseminate
29460 yak
29461 prodesigner
29462 wichever
29463 battery_charger
29464 timbre
29465 lapis
29466 longjmpe
29467 weschler
29468 accompanying
29469 availabilty
29470 beverly_hill
29471 bikini
29472 ceteris
29473 clearence
29474 conclusory
29475 convienient
29476 coud
29477 coutie
29478 gun_ownership
29479 historie
29480 homelessness
29481 inferential
29482 intercity
29483 liklihood
29484 methodoligical
29485 misstate
29486 momentus
29487 muffin
29488 noncomparable
29489 nonexperimental
29490 nonwhite
29491 paribus
29492 pervalence
29493 proverty
29494 reate
29495 se

31112 pasta
31113 pelvic
31114 ricotta
31115 sauerkraut
31116 sean_mcmain
31117 timecode
31118 manufactered
31119 dts
31120 laffer
31121 stockman
31122 cubs_suck
31123 biter
31124 nad
31125 precent
31126 sniping
31127 yelling
31128 bnn_post
31129 fathertree
31130 innocently
31131 press_briefe
31132 kilowatt
31133 backbite
31134 irreligious
31135 signpost
31136 underfoot
31137 uprightly
31138 xenophobic
31139 changeable
31140 decisiveness
31141 egocentric
31142 exhibitonist
31143 feeler
31144 histrionic
31145 humanitie
31146 pediatic
31147 playful
31148 poop
31149 psychiatry
31150 typology
31151 ance
31152 ignor
31153 unreservedly
31154 farewell
31155 zamboni
31156 aggresive
31157 toothpaste
31158 bitplaned
31159 unhelpful
31160 vord
31161 grasshopper
31162 refinish
31163 anterior
31164 degenerative
31165 fibrillation
31166 immunologic
31167 impairment
31168 ligament
31169 musculoskeletal
31170 neuromuscular
31171 pathologic
31172 pps
31173 reinnervate
31174 reinnervation
31175 cambodia

32203 ivsuffere
32204 rhinoscopy
32205 caret
32206 unencumbered
32207 ove
32208 rightmost
32209 necesarry
32210 exemplified
32211 glute
32212 mufler
32213 sargeant
32214 sigificant
32215 buttonpressmask
32216 fubare
32217 plot_data
32218 immensly
32219 inconsistently
32220 nishika
32221 conduction
32222 denervation
32223 diahrea
32224 fiornal
32225 gastritis
32226 herniation
32227 hysterectomy
32228 radiculopathy
32229 toughe
32230 uterus
32231 livid
32232 exobiologist
32233 landform
32234 tim_ciceran
32235 allotment
32236 pkunzip
32237 promblem
32238 stong
32239 intimidating
32240 lidded
32241 lotta
32242 swagger
32243 unease
32244 nastly
32245 ballad
32246 cannibal
32247 morissey
32248 remix
32249 remixes
32250 transvision
32251 vamp
32252 autobox
32253 fryer
32254 plao
32255 renumerance
32256 waii_com
32257 powerboat
32258 sailboat
32259 transom
32260 binded
32261 egotistic
32262 nurturing
32263 only
32264 penance
32265 unceasingly
32266 axg
32267 ceejz
32268 cqd
32269 eqwxl
32270 f

In [31]:
# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
#print(corpus[1:2])
corpus

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 5),
  (5, 1),
  (6, 1),
  (7, 2),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 2),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1)],
 [(3, 2),
  (6, 2),
  (13, 1),
  (35, 1),
  (39, 1),
  (40, 1),
  (41, 2),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 4),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 2),
  (54, 1),
  (55, 2),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 3),
  (69, 1),
  (70, 1),
  (71, 1),
  (72, 1),
  (73, 1),
  (74, 1),
  (75, 1),
  (76, 1),
  (77, 2),
  (78, 1),
  (79, 1),
  (80, 1),
  (81, 2),
  (82, 1)],
 [(5, 3),
  (6, 1),
  (14, 1),
  (15, 1),
  (17, 2),


In [32]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('addition', 1),
  ('body', 1),
  ('bring', 1),
  ('call', 1),
  ('car', 5),
  ('could', 1),
  ('day', 1),
  ('door', 2),
  ('early', 1),
  ('engine', 1),
  ('enlighten', 1),
  ('funky', 1),
  ('history', 1),
  ('host', 1),
  ('info', 1),
  ('know', 1),
  ('late', 1),
  ('line', 1),
  ('look', 2),
  ('mail', 1),
  ('make', 1),
  ('model', 1),
  ('name', 1),
  ('neighborhood', 1),
  ('nntp_poste', 1),
  ('park', 1),
  ('production', 1),
  ('rac_wam', 1),
  ('really', 1),
  ('rest', 1),
  ('see', 1),
  ('separate', 1),
  ('small', 1),
  ('spec', 1),
  ('sport', 1),
  ('thank', 1),
  ('where', 1),
  ('wonder', 1),
  ('year', 1)]]

## Latent Dirichlet Allocation

![Latent Dirichlet Allocation](https://miro.medium.com/max/1400/1*fCc0JT3W-1ViYyw0hJ7rdA.jpeg)

In [33]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [35]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())

[(0,
  '0.036*"write" + 0.030*"line" + 0.028*"would" + 0.021*"know" + 0.020*"be" + '
  '0.020*"go" + 0.018*"organization" + 0.017*"get" + 0.016*"host" + '
  '0.015*"article"'),
 (1,
  '0.042*"people" + 0.036*"law" + 0.033*"government" + 0.028*"child" + '
  '0.027*"kill" + 0.022*"death" + 0.021*"state" + 0.021*"right" + '
  '0.017*"country" + 0.015*"attack"'),
 (2,
  '0.115*"key" + 0.037*"chip" + 0.033*"encryption" + 0.024*"distribution" + '
  '0.023*"security" + 0.023*"engine" + 0.022*"algorithm" + 0.020*"block" + '
  '0.019*"public" + 0.019*"use"'),
 (3,
  '0.187*"space" + 0.059*"launch" + 0.047*"orbit" + 0.039*"mission" + '
  '0.039*"satellite" + 0.038*"vote" + 0.037*"earth" + 0.033*"flight" + '
  '0.028*"schedule" + 0.026*"shuttle"'),
 (4,
  '0.024*"report" + 0.022*"science" + 0.021*"patient" + 0.018*"technology" + '
  '0.017*"study" + 0.017*"research" + 0.014*"product" + 0.013*"datum" + '
  '0.013*"provide" + 0.012*"effect"'),
 (5,
  '0.075*"die" + 0.055*"season" + 0.044*"brain" + 

In [38]:
doc_lda = lda_model[corpus]

In [39]:
#Perplejidad
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Score de coherencia
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -12.278616841308214

Coherence Score:  0.47785593897129075


In [40]:
##### Visualizamos los temas
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.329751 -0.131827       1        1  30.344206
11    -0.326302 -0.160514       2        1  23.094141
12    -0.243589  0.170632       3        1   9.352917
10    -0.170717 -0.000636       4        1   6.324313
19     0.043414  0.020790       5        1   5.710670
1     -0.115727 -0.159980       6        1   4.414691
4     -0.131379  0.117948       7        1   3.970869
9     -0.068912  0.037349       8        1   3.209080
7      0.060094 -0.072084       9        1   2.110111
8     -0.001782  0.250970      10        1   2.038547
2     -0.024926  0.224316      11        1   1.941601
16     0.099432 -0.027250      12        1   1.428082
6      0.114616 -0.012908      13        1   1.231924
15     0.124839 -0.027352      14        1   1.064521
5      0.129760 -0.034608      15        1   0.964719
17     0.150819 -0.038396      16        1   0.818114
14     0.153209 -0.016865      17        1   0.742232
3      0.161995 -0.038542      18        1   0.586764
13     0.174362 -0.061456      19        1   0.417526
18     0.200547 -0.039587      20        1   0.234966, topic_info=           Term          Freq         Total Category  logprob  loglift
3700         ax  52208.000000  52208.000000  Default  30.0000  30.0000
200       write  13238.000000  13238.000000  Default  29.0000  29.0000
140      people   7560.000000   7560.000000  Default  28.0000  28.0000
38         year   4725.000000   4725.000000  Default  27.0000  27.0000
17         line  11988.000000  11988.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
4645   somewhat      0.041981      1.364621  Topic20 -11.1121   2.5721
4331    install      0.041981      1.364637  Topic20 -11.1121   2.5720
3202    digital      0.041980      1.364631  Topic20 -11.1121   2.5720
6202  president      0.041980      1.364623  Topic20 -11.1122   2.5720
3175     accord      0.041980      1.364671  Topic20 -11.1122   2.5720

[850 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1212      2  0.998981    accept
583       7  0.996898  activity
40        5  0.989354   adapter
41        1  0.139306       add
41        4  0.389857       add
...     ...       ...       ...
200       1  0.997483     write
200       2  0.002417     write
441      15  0.990523   writing
38        2  0.217335      year
38        4  0.782574      year

[1100 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 12, 13, 11, 20, 2, 5, 10, 8, 9, 3, 17, 7, 16, 6, 18, 15, 4, 14, 19])